In [1]:
############################################
# 0. 라이브러리 설치 (최초 1회)
############################################

!pip install -U openai pandas scikit-learn tqdm python-dotenv



############################################
# 1. 기본 설정
############################################
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import openai
import logging
from dotenv import load_dotenv
load_dotenv()

client = openai
openai.api_key = os.getenv("OPENAI_API_KEY")

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

if not openai.api_key:
    logging.error("OpenAI API 키가 설정되어 있지 않습니다.")
    exit(1)


############################################
# 2. CSV 로드
############################################
FILE_PATH = "D:\git_rk\data\survey.csv"
TEXT_COL = "answer"

df = pd.read_csv(FILE_PATH)
df = df[df[TEXT_COL].notna()].copy()
df.rename(columns={TEXT_COL: "원본"}, inplace=True)

texts = df["원본"].astype(str).tolist()
print(f"총 {len(texts)}건 주관식 응답 로드 완료")


############################################
# 3. 원본 → 요약 (왜곡 금지)
############################################
def summarize_row(text):
    prompt = f"""
다음은 축제(행사)에 대한 설문 주관식 응답입니다.

⚠️ 매우 중요:
- 원문에 없는 내용은 절대 작성하지 마세요.
- 추론, 일반화, 의미 확장 금지
- '요약', '요약:' 같은 라벨이나 제목을 절대 출력하지 마세요.
- 출력은 내용만 작성하세요.

아래 응답에서
응답자가 직접 언급한 핵심 내용만
한 줄로 자연스럽게 정리해 주세요.

설문 응답:
{text}

출력:
"""
    r = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": "설문 응답 요약 전문가"},
            {"role": "user", "content": prompt}
        ]
    )
    return r.choices[0].message.content.strip()


summaries = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for result in tqdm(executor.map(summarize_row, texts), total=len(texts)):
        summaries.append(result)

df["요약"] = summaries


############################################
# 4. 요약 → 요약 개조식 (7단어 이내)
############################################
def bulletize_summary(summary):
    prompt = f"""
아래 문장은 설문 응답 요약 문장입니다.

⚠️ 규칙:
- 아래 문장에 등장한 단어와 표현만 사용
- 새로운 단어, 의미, 해석 추가 금지
- 단어 수는 최대 7개 이내
- 조사 최소화
- 라벨(요약, 정리 등) 출력 금지
- 개조식 한 줄로만 출력

요약 문장:
{summary}

출력:
- 
"""
    r = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": "설문 요약 축약 전문가"},
            {"role": "user", "content": prompt}
        ]
    )
    return r.choices[0].message.content.strip()


bullet_summaries = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for result in tqdm(executor.map(bulletize_summary, df["요약"].tolist()), total=len(df)):
        bullet_summaries.append(result)

df["요약 개조식"] = bullet_summaries


############################################
# 5. 요약 → 카테고리화 (임베딩 + 클러스터링)
############################################
def get_embeddings_batch(texts, model="text-embedding-3-small", batch_size=50):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        r = client.embeddings.create(model=model, input=batch)
        all_embeddings.extend([d.embedding for d in r.data])
    return np.array(all_embeddings)

summary_embeddings = get_embeddings_batch(df["요약"].tolist())

N_CLUSTERS = 5
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42)
df["cluster_id"] = kmeans.fit_predict(summary_embeddings)


############################################
# 6. 클러스터 → 카테고리명 생성
############################################
def generate_cluster_name(texts):
    sample = "\n".join(texts[:10])
    prompt = f"""
다음 요약 문장 묶음을 대표하는 카테고리명을 만들어 주세요.

조건:
- 요약 문장에 실제 등장한 표현 기반
- 추론·확장 금지
- 10자 내외 명사형

요약 문장:
{sample}

출력:
"""
    r = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": "설문 데이터 분류 전문가"},
            {"role": "user", "content": prompt}
        ]
    )
    return r.choices[0].message.content.strip()

cluster_names = {}
for c in sorted(df["cluster_id"].unique()):
    texts_c = df[df["cluster_id"] == c]["요약"].tolist()
    cluster_names[c] = generate_cluster_name(texts_c)

df["카테고리"] = df["cluster_id"].map(cluster_names)


############################################
# 7. 최종 산출물 저장
############################################
base_filename = os.path.splitext(os.path.basename(FILE_PATH))[0]
now_str = datetime.now().strftime("%Y%m%d_%H%M")

excel_path = f"data/{base_filename}_{now_str}.xlsx"
csv_path   = f"data/{base_filename}_{now_str}.csv"

final_df = df[["원본", "요약", "요약 개조식", "카테고리"]]
display(final_df.head())

final_df.to_excel(excel_path, index=False)
final_df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"\n✅ 엑셀 저장 완료: {excel_path}")
print(f"✅ CSV 저장 완료: {csv_path}")


<>:37: SyntaxWarning: "\g" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\g"? A raw string is also an option.
<>:37: SyntaxWarning: "\g" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\g"? A raw string is also an option.
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_37596\525095009.py:37: SyntaxWarning: "\g" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\g"? A raw string is also an option.
  FILE_PATH = "D:\git_rk\data\survey.csv"


총 3건 주관식 응답 로드 완료


  0%|          | 0/3 [00:00<?, ?it/s]2026-01-05 21:31:09,507 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-05 21:31:10,226 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 3/3 [00:38<00:00, 12.71s/it]
2026-01-05 21:31:52,844 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_37596\525095009.py:37: SyntaxWarning: "\g" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\g"? A raw string is also an option.
  FILE_PATH = "D:\git_rk\data\survey.csv"


ValueError: n_samples=3 should be >= n_clusters=5.

None
